# Mossley Hill Church: Geographic Data Science Internship

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import geopandas as gpd
import geoplot
import os.path
import osmnx as ox
import networkx as nx
import requests
import json

import zipfile
import io

In [2]:
church_postcode = 'L18 8DB'
church_coords = (53.376628, -2.920006)

In [3]:
one_mile = 1609 #meters
immediate_neighbourhood = ox.graph_from_point(church_coords, distance=1*one_mile, network_type='drive')

In [4]:
def get_postcode_coords(postcodes):
    postcode_data = {}
    for postcode in postcodes:
        url = 'https://api.postcodes.io/postcodes/'
        postcode_data[postcode] = json.loads(requests.get(url+'/'+postcode).text)
    coords = {}
    for postcode in postcodes:
        try:
            coords[postcode] = postcode_data[postcode]['result']['latitude'], postcode_data[postcode]['result']['longitude']
        except:
            coords[postcode] = 'not found'
    return coords

## The Congregation

In [5]:
electoral_roll = pd.read_csv(os.path.join('..','mh-church-data','electoral-roll.csv'))

FileNotFoundError: [Errno 2] File data/electoral-roll.csv does not exist: 'data/electoral-roll.csv'

In [ ]:
len(electoral_roll)

There are 242 people on the church electoral roll.

In [ ]:
ages = pd.read_excel(os.path.join('..','mh-church-data','mossley-hill-numbers.xlsx'), sheet_name='Regulars', index_col=1)

In [ ]:
ages[0:6].plot.bar()

In [ ]:
electoral_roll.Sex.value_counts()

There are almost twice as many women as men.

In [ ]:
electoral_roll.Sex.hist(grid=False, figsize=(1,5))

In [ ]:
postcodes = list(electoral_roll.Postcode.unique())

In [ ]:
len(postcodes)

There are 149 unique postcodes. Families (within a household) must share a postcode, though not everyone who shares a postcode is necessarily in the same household. If we assume that everyone sharing a postcode is part of the same household, then there might be 18 families with a couple and one or more children.

In [ ]:
electoral_roll.Postcode.value_counts()[0:20]

In [ ]:
coords = get_postcode_coords(postcodes)

In [ ]:
one_mile = 1609 #meters

In [ ]:
# G = ox.graph_from_point(church_coords, distance=10*one_mile, network_type='drive')
# # fig, ax = ox.plot_graph(G, fig_height=20, node_size=0)

# # save graph to disk as shapefile (for GIS) or graphml file (for gephi etc)
# ox.save_graph_shapefile(G, filename='mynetwork_shapefile')
# ox.save_graphml(G, filename='mynetwork.graphml')

In [ ]:
G = ox.load_graphml('mynetwork.graphml')

In [ ]:
dest_node = ox.get_nearest_node(G, church_coords)
shortest_path = {}
distance = {}
for postcode in postcodes:
    
    try:
        orig_node = ox.get_nearest_node(G, coords[postcode])
        shortest_path[postcode] = nx.shortest_path(G, orig_node, dest_node, weight='length')
        # how long is our route in meters?
        distance[postcode] = nx.shortest_path_length(G, orig_node, dest_node, weight='length') / one_mile
    except:
        pass

In [ ]:
all_routes = []
for postcode in postcodes:
    try:
        all_routes.append(shortest_path[postcode])
    except:
        pass

In [ ]:
nearby = {}
nearby_postcodes = []
nearby_routes = []
for postcode in postcodes:
    try:
        if distance[postcode] < 1:
            nearby_postcodes.append(postcode)
            nearby_routes.append(shortest_path[postcode])
    except:
        pass

In [ ]:
len(nearby_postcodes)

In [ ]:
fig, ax = ox.plot_graph_routes(immediate_neighbourhood, nearby_routes, fig_height=10, node_size=0, orig_dest_node_size=10, edge_alpha=0.3, route_alpha=0.1, route_linewidth=1, show=False, close=False)
# for p in church_postcodes:
#     c = other_church_coords[p]
#     ax.scatter(c[1],c[0],c = 'blue')
ax.scatter(church_coords[1], church_coords[0], c='red')
plt.show()

In [ ]:
# plot the routes https://osmnx.readthedocs.io/en/stable/osmnx.html#module-osmnx.plot
fig, ax = ox.plot_graph_routes(G2, all_routes, fig_height=20, node_size=0, orig_dest_node_size=10, edge_alpha=0.3, route_linewidth=1, show=False, close=False)
# for p in church_postcodes:
#     c = other_church_coords[p]
#     ax.scatter(c[1],c[0],c = 'blue')
ax.scatter(church_coords[1], church_coords[0], c='red')
plt.show()

In [ ]:
dist = pd.DataFrame(distance, index=['Distance']).transpose()

electoral_roll.set_index('Postcode', inplace=True)

electoral_roll.insert(loc=1,column='Distance',value=dist.Distance)

In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()
ax.set_ylim(0,60)
ax2.set_ylim(0,1)
ax.set_xlim(0,12)
electoral_roll.Distance.hist(bins=50, ax=ax)
electoral_roll.Distance.plot.kde(ax=ax2, c = 'r')

In [ ]:
electoral_roll.Distance.describe()

In [ ]:
print('Furthest')
electoral_roll.sort_values('Distance', ascending=False)[0:20].transpose()

In [ ]:
print('Nearest')
electoral_roll.sort_values('Distance',)[0:20].transpose()

In [ ]:
electoral_roll.loc[electoral_roll.Distance.isnull()].transpose()

## The Parish

[Mossley Hill Church](https://mossleyhillchurch.org.uk/who-we-are/), Rose Lane, Liverpool L18 8DB.

>The Church of St Matthew and St James stands at the top of the hill in Rose Lane, Mossley Hill, Liverpool. It is an active Anglican parish church in the deanery of Liverpool South Childwall, the archdeaconry of Liverpool and the diocese of Liverpool. The church is recorded in the National Heritage List for England as a designated Grade II* listed building. The authors of the Buildings of England series describe it as "one of the best Victorian churches in Liverpool"

In [ ]:
churches = pd.read_csv(os.path.join('data','other-churches.csv'))

In [ ]:
church_postcodes = list(churches.Postcode)

In [ ]:
other_church_coords = get_postcode_coords(church_postcodes)

In [ ]:
churchlist = ''
for church in churches.Church:
    churchlist += church + '; '

print(f'**Nearby Churches**: {churchlist[:-2]}')

In [ ]:
churches.transpose()

In [ ]:
fig, ax = ox.plot_graph(immediate_neighbourhood, fig_height=10, node_size=0, show=False, close=False)
for p in church_postcodes:
    c = other_church_coords[p]
    ax.scatter(c[1],c[0],c = 'blue')
ax.scatter(church_coords[1], church_coords[0], c='red', s=100)
plt.show()

In [ ]:
def get_street_names(coords, miles):
    neighbourhood_graph = ox.project_graph(ox.graph_from_point(coords, distance=miles*one_mile, network_type='drive'))
    ints = ox.clean_intersections(neighbourhood_graph)

    gdf = gpd.GeoDataFrame(ints, columns=['geometry'], crs=neighbourhood_graph.graph['crs'])
    X = gdf['geometry'].map(lambda pt: pt.coords[0][0])
    Y = gdf['geometry'].map(lambda pt: pt.coords[0][1])

    nodes = ox.get_nearest_nodes(neighbourhood_graph, X, Y, method='kdtree')
    nearby_streets = {'Rose Lane'}
    for n in nodes:
        for nbr in nx.neighbors(neighbourhood_graph, n):
            for d in neighbourhood_graph.get_edge_data(n, nbr).values():
                if 'name' in d:
                    if type(d['name']) == str:
                        nearby_streets.add(d['name'])
                    elif type(d['name']) == list:
                        for name in d['name']:
                            nearby_streets.add(name)
                    else:
                        pass
                else:
                    pass

    return nearby_streets

In [ ]:
miles = 0.5
nearby_streets = get_street_names(church_coords, miles)
print(f'There are {len(nearby_streets)} streets within {miles} miles of the church.')
print(nearby_streets)

## Liverpool Neighbourhoods

In [ ]:
# get neighbourhood data

In [ ]:
oac2011 = pd.read_csv(os.path.join('data','2011 OAC Clusters and Names csv v2.csv'))

In [ ]:
oac2011['Region/Country Name'].unique()

In [ ]:
northwest = oac2011.loc[oac2011['Region/Country Name']=='North West']

northwest.rename(columns={'Output Area Code':'OA11CD'}, inplace=True)

In [ ]:
northwest['Local Authority Name'].unique()

In [ ]:
liverpool = northwest.loc[northwest['Local Authority Name']=='Liverpool']

In [ ]:
# shape_url = 'https://opendata.arcgis.com/datasets/ff8151d927974f349de240e7c8f6c140_0.zip'

# shape_data = gpd.read_file(shape_url)
# shape_data.plot()

In [ ]:
liverpool_shapes = shape_data.merge(liverpool, on='OA11CD')
liverpool_shapes.to_file(os.path.join('data','liverpool.shp'))

In [ ]:
liverpool_from_file = gpd.read_file(os.path.join('data','liverpool.shp'))

In [ ]:
liverpool_from_file.plot()

In [ ]:
nw_shapes['Supergroup Code'].unique()

In [ ]:
supergroups = list(oac2011['Supergroup Name'].unique())
supergroups

In [ ]:
clusters = range(8)

# https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.plot.plot_graph_routes
fig, ax = ox.plot_graph_routes(immediate_neighbourhood, nearby_routes, fig_height=10, node_size=0, orig_dest_node_size=500, orig_dest_node_color='k',edge_alpha=1, route_alpha=.1, route_color='k', route_linewidth=10, show=False, close=False)

nw_clusters = {}
for c in clusters:
    nw_clusters[c] = liverpool_shapes[liverpool_shapes['Supergroup Code']==c+1]

# these colours are ugly, but I can change them later
# i've tried to make them match the standard https://maps.cdrc.ac.uk/#/geodemographics/oac11/, 
cluster_colors = ['green', 'red', 'pink', 'orange', 'brown', 'purple', 'blue',  'yellow']

for c in clusters:
    nw_clusters[c].plot(color=cluster_colors[c], ax=ax)
    ax.set_facecolor('#e0f8f8')
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])

fig.suptitle('Figure: Mossley Hill Geodemographic Visualization',
            fontsize=16,
            y=0.15, x = 0.17)
fig.tight_layout()
ax.scatter(church_coords[1], church_coords[0], c='black', s=500)

plt.show()

In [ ]:
convert_url = 'https://www.arcgis.com/sharing/rest/content/items/ef72efd6adf64b11a2228f7b3e95deea/data'
r = requests.get(convert_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
filename = 'PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.csv'
conversion_chart = pd.read_csv(z.open(filename))

In [ ]:
conversion_chart.set_index('PCD7', inplace=True)

In [ ]:
lp_pc2oa = conversion_chart[['OA11CD']].loc[conversion_chart['LAD11NM']=='Liverpool']

In [ ]:
conversion_dict = lp_pc2oa.transpose().to_dict()

In [ ]:
liverpool.set_index('OA11CD',inplace=True)

In [ ]:
oac_dict = liverpool[['Supergroup Name']].transpose().to_dict()

In [ ]:
nbhd = {}
for p in postcodes:
    try:
        n = oac_dict[conversion_dict[p]['OA11CD']]['Supergroup Name']
    except:
        n = 'unknown'
    nbhd[p] = n

In [ ]:
nbhd_df = pd.DataFrame(nbhd, index=['Type']).transpose()
electoral_roll.insert(loc=1,column='Neighbourhood Type',value=nbhd_df.Type)

In [ ]:
electoral_roll['Neighbourhood Type'].value_counts()

In [ ]:
electoral_roll['Neighbourhood Type'].value_counts().plot.bar()

> **Suburbanites** The population of this supergroup is most likely to be located on the outskirts of urban areas. They
are more likely to own their own home and to live in semi-detached or detached properties. The
population tends to be a mixture of those above retirement age and middle-aged parents with
school age children. The number of residents who are married or in civil-partnerships is above the
national average. Individuals are likely to have higher-level qualifications than the national average,
with the levels of unemployment in these areas being below the national average. All non-White
ethnic groups have a lower representation when compared with the UK and the proportion of
people born in the UK or Ireland is slightly higher. People are more likely to work in the information
and communication, financial, public administration, and education sectors, and use private
transport to get to work. [OAC2011 Pen Portraits](https://www.ons.gov.uk/file?uri=/methodology/geography/geographicalproducts/areaclassifications/2011areaclassifications/penportraitsandradialplots/penportraits.pdf)

## Other Neighbourhood Data

In [ ]:
# annual age/population estimates
age_url = 'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fpopulationandmigration%2fpopulationestimates%2fdatasets%2fcensusoutputareaestimatesinthenorthwestregionofengland%2fmid2018sape21dt10b/sape21dt10bmid2018northwest.zip'
r = requests.get(age_url)
r.content
z = zipfile.ZipFile(io.BytesIO(r.content))
filename = 'SAPE21DT10b-mid-2018-coa-unformatted-syoa-estimates-north-west.xlsx'
age_data = pd.read_excel(z.open(filename), sheet_name = 'Mid-2018 Persons')
male_age_data = pd.read_excel(z.open(filename), sheet_name = 'Mid-2018 Males')
female_age_data = pd.read_excel(z.open(filename), sheet_name = 'Mid-2018 Females')

In [ ]:
# deprivation
# https://www.gov.uk/government/statistics/english-indices-of-deprivation-2019

In [ ]:
# schools
## https://www.compare-school-performance.service.gov.uk/download-data

In [ ]:
# house prices
## https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/medianpricepaidbylowerlayersuperoutputareahpssadataset46

In [ ]:
# income
## https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/smallareaincomeestimatesformiddlelayersuperoutputareasenglandandwales

In [ ]:
# 2011 census data
## https://www.nomisweb.co.uk/census/2011/bulk/r2_2